## Get Relevant Paragraphs

In [9]:
import json

with open('strategyqa-data/strategyqa_dataset/strategyqa_train.json', 'r') as f:
    qa = json.load(f)
    
question_set = set()
with open('question_clusters/cluster4.json', 'r') as f:
    loaded_strings = json.load(f)
    
for l in loaded_strings:
    question_set.add(l)

In [10]:
relevant_qas = []
for q in qa:
    if q['question'] in question_set:
        relevant_qas.append(q)

In [26]:
from collections.abc import Iterable

def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [65]:
relevant_paragraphs = set([])
for q in relevant_qas:
    relevant_paragraphs = relevant_paragraphs | set(flatten([q['evidence']]))
    
# two specific calls to strategyQA
relevant_paragraphs.remove("no_evidence")
relevant_paragraphs.remove("operation")
    
print("There are {} relevant paragraphs".format(len(relevant_paragraphs)))

There are 780 relevant paragraphs


## Total Number of Wikipedia Pages

In [58]:
removed_paragraph_endings = set([])
for p in relevant_paragraphs:
    removed_paragraph_endings.add("-".join(p.split("-")[:-1]))
print("Which come from {} pages".format(len(removed_paragraph_endings)))

Which come from 472 pages


In [52]:
count_p = 0
num_lines = 36617357
with open('./strategyqa-data/corpus-enwiki-20200511-cirrussearch-parasv2.jsonl', 'r') as file:
    for line in tqdm(file, total=num_lines):
        obj = json.loads(line)

        # Check if the 'title' parameter of the object exists in the given set
        if 'title' in obj and obj['title'] in removed_paragraph_endings:
            count_p += 1

100%|██████████████████████████████████████████| 36617357/36617357 [01:35<00:00, 385201.88it/s]


In [60]:
print("Overall there are {} related pages in total".format(count_p))

Overall there are 21396 related pages in total


The 782 paragraphs will be used for training. Ideally, all 21k can be converted and stored in Qdrant

In [71]:
with open('strategyqa-data/strategyqa_dataset/strategyqa_train_paragraphs.json', 'r') as f:
    paragraph_content = json.load(f)

paragraphs_to_index = []
for p in relevant_paragraphs:
    paragraphs_to_index.append(paragraph_content[p]['content'])

In [72]:
with open('paragraph_content.json', 'w') as f:
    json.dump(paragraphs_to_index, f)